#Refer https://github.com/ritchieng/deep-learning-wizard/blob/master/docs/deep_learning/practical_pytorch/pytorch_lstm_neuralnetwork.ipynb



In [16]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets

train_dataset = dsets.MNIST(root='./data',
                            train=True,
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data',
                           train=False,
                           transform=transforms.ToTensor())


In [17]:
print(train_dataset.train_data.size())
print(train_dataset.train_labels.size())
print(test_dataset.test_labels.size())
print(test_dataset.test_data.size())

torch.Size([60000, 28, 28])
torch.Size([60000])
torch.Size([10000])
torch.Size([10000, 28, 28])


In [18]:
batch_size = 100
n_iters = 3000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)
print(num_epochs)

5


In [19]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim

        # Number of hidden layers
        self.layer_dim = layer_dim

        # Building your LSTM
        # batch_first=True causes input/output tensors to be of shape
        # (batch_dim, seq_dim, feature_dim)
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)

        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        # Initialize cell state
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        # 28 time steps
        # We need to detach as we are doing truncated backpropagation through time (BPTT)
        # If we don't, we'll backprop all the way to the start even after going through another batch
        print("X Shape", x.shape)
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))

        # Index hidden state of last time step
        # out.size() --> 100, 28, 100
        # out[:, -1, :] --> 100, 100 --> just want last time step hidden states!
        out = self.fc(out[:, -1, :])
        # out.size() --> 100, 10
        return out

In [20]:
input_dim = 28
hidden_dim = 100
layer_dim = 2
output_dim = 10

model = LSTMModel(input_dim, hidden_dim, layer_dim, output_dim)

In [21]:
criterion = nn.CrossEntropyLoss()

In [22]:
learning_rate = 0.1

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [23]:
len(list(model.parameters()))

10

In [24]:
for i in range(len(list(model.parameters()))):
    print(list(model.parameters())[i].size())

torch.Size([400, 28])
torch.Size([400, 100])
torch.Size([400])
torch.Size([400])
torch.Size([400, 100])
torch.Size([400, 100])
torch.Size([400])
torch.Size([400])
torch.Size([10, 100])
torch.Size([10])


In [27]:
# Number of steps to unroll
seq_dim = 28

iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Load images as a torch tensor with gradient accumulation abilities
        #print("Image Shape before", images.shape)

        images = images.view(-1, seq_dim, input_dim).requires_grad_()

        #print("Image Shape after", images.shape)
        # Clear gradients w.r.t. parameters

        optimizer.zero_grad()

        # Forward pass to get output/logits
        # outputs.size() --> 100, 10
        outputs = model(images)

        # Calculate Loss: softmax --> cross entropy loss
        print("labels shape",labels.shape)
        loss = criterion(outputs, labels)

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()

        iter += 1

        if iter % 500 == 0:
            # Calculate Accuracy
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                # Resize images
                images = images.view(-1, seq_dim, input_dim)

                # Forward pass only to get logits/output
                outputs = model(images)

                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)

                # Total number of labels
                total += labels.size(0)

                # Total correct predictions
                correct += (predicted == labels).sum()

            accuracy = 100 * correct / total

            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))


X Shape torch.Size([100, 28, 28])
labels shape torch.Size([100])
X Shape torch.Size([100, 28, 28])
labels shape torch.Size([100])
X Shape torch.Size([100, 28, 28])
labels shape torch.Size([100])
X Shape torch.Size([100, 28, 28])
labels shape torch.Size([100])
X Shape torch.Size([100, 28, 28])
labels shape torch.Size([100])
X Shape torch.Size([100, 28, 28])
labels shape torch.Size([100])
X Shape torch.Size([100, 28, 28])
labels shape torch.Size([100])
X Shape torch.Size([100, 28, 28])
labels shape torch.Size([100])
X Shape torch.Size([100, 28, 28])
labels shape torch.Size([100])
X Shape torch.Size([100, 28, 28])
labels shape torch.Size([100])
X Shape torch.Size([100, 28, 28])
labels shape torch.Size([100])
X Shape torch.Size([100, 28, 28])
labels shape torch.Size([100])
X Shape torch.Size([100, 28, 28])
labels shape torch.Size([100])
X Shape torch.Size([100, 28, 28])
labels shape torch.Size([100])
X Shape torch.Size([100, 28, 28])
labels shape torch.Size([100])
X Shape torch.Size([100, 

KeyboardInterrupt: 